<div class = "intro">

# Introduction to Machine Learning for Materials Science
## Lab Activity

<div class = "cellContent">
    
   
This lab introduces students to an end-to-end example of applying a machine learning (ML) workflow to a materials science dataset to address a research problem. The lab aims at deepening the conceptual understanding of ML, and while procedural skills such as writing Python code are not the focus of this lab, students will gain experience with a number of standard open source packages by interacting with code snippets through the Jupyter Notebook format and describing what each essential command does.


</div> <!-- closes content -->

</div> <!-- closes intro -->



<div class = "section toc">


## Table of Contents<a name="toc"></a>

<div class = "cellContent">
    
- ### [Lesson Format](#gs)
- ### [Jupyter Notebooks Tips and Tricks](#jn)
- ### [Section 0: Setup](#0)
- ### [Section 1: Data Inspection](#1)
    - ### [Importing Dataset](#lesson1.1)
        - #### [Exercise 1.1](#e1.1)
    - ### [Data Cleaning](#lesson1.2) 
        - #### [Exercise 1.2](#e1.2)
        - #### [Exercise 1.3](#e1.3)
    - ### [Evaluating Data Availability](#lesson1.3)
        - #### [Exercise 1.4](#e1.4)
        - #### [Exercise 1.5](#e1.5)
        - #### [Exercise 1.6](#e1.6)
- ### [Section 2: Feature Generation](#2)
    - ### [MASTML Configuration](#lesson2.1)
    - ### [Understanding Compositional Average Features](#lesson2.2)
        - #### [Exercise 2.1](#e2.1)
- ### [Section 3: Feature Engineering](#3)
    - ### [Remove Constant Columns](#lesson3.1)
        - #### [Exercise 3.1](#e3.1)
    - ### [Remove Highly Correlated Columns](#lesson3.2)
        - #### [Exercise 3.2](#e3.2)
    - ### [Feature Normalization](#lesson3.3)
- ### [Section 4: Setup for Model Evaluation](#4)
    - ### [Establishing train/test split](#lesson4.1)
    - ### [Evaluating train/test split](#lesson4.2)
        - #### [Exercise 4.1](#e4.1)
- ### [Section 5: Fitting and Evaluating a Default Model](#5)
    - ### [Fitting the Decision Tree Model](#lesson5.1)
    - ### [Evaluating Model Performance on Training Data](#lesson5.2)
        - #### [Exercise 5.1](#e5.1)
    - ### [Evaluating Model Performance on Test Data](#lesson5.3)
        - #### [Exercise 5.2](#e5.2)
- ### [Section 6: Hyperparameter Optimization](#6)
    - ### [Establishing a cross-validation scheme](#lesson6.1)
    - ### [Defining a parameter space](#lesson6.2)
    - ### [Setting up a grid search](#lesson6.3)
        - #### [Exercise 6.1](#e6.1)
    - ### [Visualizing bias-variance tradeoff](#lesson6.4)
        - #### [Exercise 6.2](#e6.2)
    - ### [Default vs. optimized model: training and validation data performance](#lesson6.5)
        - #### [Exercise 6.3](#e6.3)
    - ### [Default vs. optimized model: test data performance](#lesson6.6)
        - #### [Exercise 6.4](#e6.4)
        - #### [Exercise 6.5](#e6.5)
    - ### [Default vs. optimized model: visualizing the decision trees](#lesson6.7)
        - #### [Exercise 6.6](#e6.7)
        - #### [Exercise 6.7](#e6.7)
- ### [Section 7: Make Predictions](#7)
    - #### [Exercise 7.1](#e7.1)

</div> <!--end content-->

</div> <!--end section-->



<div class = "button">

[\[-----------------------------------Begin Lesson-----------------------------------\]](#0) 
    
</div>

<div class = "section">

# Setup<a name="0"></a>
---
[Back to TOC](#toc)

<div class = "cellContent">

## Learning Outcomes
1. Describe the purpose of essential Python packages used in this lab
2. Recall that helper functions are available in this lab


</div> <!-- closes content -->

</div> <!-- closes section -->

<div class = "lesson">

### Python Packages<a name = "lesson0.1"></a>


<div class = "cellContent">

There are a number of Python packages we'll use throughout the lab. Each of the import statements below imports or loads these packages in so that we can use them. A brief description is given for each one.

</div> <!-- closes content -->

</div> <!-- closes lesson -->


In [1]:
import os                        # OS stands for Operating System and provides ways for python to interact with files or directories
from collections import Counter  # Collections is a package for handling data
from pprint import pprint

import pandas as pd              # Pandas is a data analysis library which we'll primarily use to handle our dataset
import numpy as np               # Numpy is a package for scientific computing. We'll use it for some of it's math functions
import pymatgen                  # Pymatgen is a library for materials analysis which we use to interpret our material compositions

In [2]:
import matplotlib                # Matplotlib is the plotting package that we'll use throughout the lab
import matplotlib.pyplot as plt
import seaborn as sns            # Seaborn is a Python data visualization library based on matplotlib

/home/mehr/anaconda3/lib/python3.11/site-packages/seaborn/_statistics.py:31: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.23.1)
  from scipy.stats import gaussian_kde


In [3]:
import sklearn                   # Scikit-learn is a machine learning package, providing the backbone for the work we'll perform
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate,GridSearchCV,ParameterGrid
from sklearn.model_selection import KFold,RepeatedKFold

In [4]:
import graphviz                  # graphviz is a package that helps visualize decision trees

ModuleNotFoundError: No module named 'graphviz'

<div class = "lesson">

### Helper Functions<a name = "lesson0.2"></a>


<div class = "cellContent">

The final import we'll do is import some custom functions that mainly put together the plots and some specific analysis that we'll use throughout the lab. If you're interested in learning what exactly these look like you can open up the helper_functions.py text file that is in the same folder as this notebook.

</div> <!-- closes content -->

</div> <!-- closes lesson -->


In [5]:
from helper_functions import *

ModuleNotFoundError: No module named 'helper_functions'

There are a number of steps we'll take which would normally have a random state. In order to have consistent results we'll fix them all by setting a random seed for all those processes.

In [6]:
seed = 2345312

<div class = "section">

# Materials Background<a name="0.5"></a>
___

[Back to TOC](#toc)

<div class = "cellContent">
    
## Band Gap Dataset
Throughout the lab we'll be working with a digitized version of a band gap dataset compiled by two scientists, W.H. Strehlow and E.L. Cook. They compiled this dataset of elementary and binary compound semiconductors in 1973 by searching through 723 individual references. Along with simply compiling reported measurements they also took the time to identify more and less reliable data points by taking "into consideration the material, the method of measurement, the reported sample purity, and the experimental conditions." They also comment that experimental measures are often greater than 5% in error and seldom less than 1%. 

https://citrination.com/datasets/1160/show_files


## Uses In Materials Science
Having access to a dataset such as this is a great place to start for machine learning. In this case band gap data has been generated over many years, and often times the first step in any machine learning problem is simply acquiring the data. Luckily Strehlow and Cook have done the largest part of the work for us already! So now let's think about how we might want to use the data that we have. We'll focus on two potential applications for our machine learning models, which will give us something to call back to when we want to decide how well the models are performing

### Predicting Materials for Single-Junction Solar Cells
Solar power is growing as a renewable energy source and there are two main factors that could improve it's viability when competing with other sources of energy. First is the cost of manufacturing materials, and the second is the efficiency of the Solar Cells themselves. If cost could be lowered via the discovery of new materials that are cheaper, and the efficiency of cells can be improved by finding materials with ideal band gaps, then solar energy could grow faster! So lets set our first goal as being a model that can predict potential solar materials based on their band gap. For simplicity we'll limit ourselves to single-junction cells, which allows us to more clearly define an operational range of band gaps. Following the suggestions from the links below (which cover things in much greater detail), in order to obtain reasonable efficiency in a single-junction solar cell we would need a material with a band gap in the range of 1.0 eV to 1.7 eV. In order to make predictions in this range then, we would need a model with an accuracy of +- 0.35 eV at the most to give us a shot and having predictions actually be within that range. 

For a more thorough background see the following:
http://solarcellcentral.com/junction_page.html
http://solarcellcentral.com/limits_page.html


### Predicting Wide Band Gap Semi-conductors
There are also a range of semi-conductor devices that are relatively less well known which fall under the category of Wide Band Gap semi-conductors (WBG). A few example devices would be in industrial motors where they can improve efficiency, in LED lighting and lasers, and in general electronics. Several properties of WBGs that give these device improvements are their higher voltage, temperature, and frequency operation ranges. For a more thorough discussion see the linked review! So again if we can build a model to predict WBG materials, we can potentially accelerate development of these next generation devices. Our criteria for predicting WBG materials will be having a bandgap between 2 eV and 4 eV. whereas more traditional semi-conductors have band gaps below 2 eV. That means for predicting WBG materials we need a model with an accuracy of +- 1 eV!

Reference: https://www1.eere.energy.gov/manufacturing/rd/pdfs/wide_bandgap_semiconductors_factsheet.pdf

</div> <!-- closes content -->

</div> <!-- closes section -->

<div class = "section">

# Data Cleaning and Inspection<a name="1"></a>
___

[Back to TOC](#toc)

<div class = "cellContent">
    
## Overview
Before getting started, let's first get to know the dataset we'll be working with a little. We'll be taking a look at what information is contained in each column, and how we can use that information to set ourselves up for success later when we start building our machine learning models. Remember that machine learning models are looking for patterns in the data! So any cleaning or pre-processing that we're doing here is with the goal of making those patterns easier to learn!

## Learning Outcomes
1. Recall common issues with materials datasets that require data cleaning
2. Recognize effect of data cleaning steps on the dataset
3. Evaluate the dataset using criteria of data cleaning
4. Evaluate feasibility of different ML applications given the available data



</div> <!-- closes content -->

</div> <!-- closes section -->

## Content/Exercises/Lessons

<div class = "lesson">

### Importing the Dataset<a name = "lesson1.1"></a>


<div class = "cellContent">

Through out this lab, you will see many objects in the code named as `_df`, which stands for <u>dataframe</u>, the primary data structure from the `pandas` package. It is similar to a spreadsheet or a table.

We won't discuss dataframe operations in detail in this lab. However, external resources are available for you to familiarize yourself with dataframes such as [this introduction](https://towardsdatascience.com/pandas-dataframe-a-lightweight-intro-680e3a212b96) and [the pandas documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

</div> <!-- closes content -->

</div> <!-- closes lesson -->


In [8]:
# Read in the band gap data from our dataset
mastml_df = pd.read_csv("bandgap_data_v2.csv")

In [9]:
mastml_df

,index,chemicalFormula Clean,Band gap values Clean,Band gap units,Band gap method,Reliability
0,0,Li1F1,13.60,eV,Reflection,1
1,1,Li1F1,12.61,eV,Reflection,1
2,2,Li1F1,12.60,eV,Estimated,2
3,3,Li1F1,12.10,eV,Absorption,2
4,4,Li1F1,12.00,eV,Absorption,2
...,...,...,...,...,...,...
1442,1454,Th1O2,3.30,eV,Reflection,2
1443,1455,UO,1.50,eV,Thermal activation,1
1444,1456,U1O2,2.18,eV,Absorption,1
1445,1457,UO,0.60,eV,Thermal activation,2


Before we dig into too much detail, lets take a second to understand what is included in the dataset column by column:

1) Index   
When dealing with large datasets having an explicit index is essential for keeping track of data points. Throughout the lab we'll be making changes to the dataset, and without proper indexing it's easy to make mistakes and lose track of where data came from.
By Specifying a unique number to each datapoint we can always track things down to troubleshoot, make later changes, or track where something came from.

2) chemicalFormula Clean  
This is the key input parameter for all of the models you'll build. Fundamentally all of the information that the model contains can be represented by the chemical formulas in this column. 
Take a second to think about how powerful it would be to have a model that only has these simple letters and numbers as input. With an accurate model it would be possible to think of any composition of interest and obtain an almost immediate prediction.

3) Band gap values Clean, Band gap units  
Carrying on from the previous thought we have to ask ourselves "what is it that we're predicting?". In this case we have a dataset of band gap values for semiconductors and insulators. Knowledge of a material's bandgap is essential for a whole range of semiconductor applications.
If we could predict a new material's band gap we could potentially accelerate discovery and design of materials, contributing to what is already more than a 400 billion dollar industry!

4) Band gap method  
We won't dive too deeply into the method information here directly, but notice that in the dataset we have a few different experimental measurement types. This is often the case when putting together large datasets that not all data is exactly equal. 
The accuracy of a model is often limited by the quality of data availabe so it's always important to understand where are the data comes from, and if it can be combined.

5) Reliability  
As a simpler version of the idea above about data quality we have a column labeled "Reliability". The researcher who put the dataset together took time to check each of their sources and come up with a reliability score or 1 or 2. 
A score of 1 indicates the most reliable data, and a 2 indicates that the samples may have been less pure, or the experimental technique was less accurate. As part of the data cleaning process later on we'll only use the most reliable data we have.

<div class = "exercise">

### Exercise 1.1<a name="e1.1"></a>

Question:  
1. Look at the first 5 rows of this table. What do you notice about their chemical formula prevents us from this dataset as-is to train a machine learning model?  

[Check or Submit Answers](#a1.1)

</div>

<div class = "lesson">

### Data cleaning<a name = "lesson1.2"></a>


<div class = "cellContent">

The bandgap dataset needs to be free of inconsistencies or ambiguities for machine learning to work properly. At this stage, the most important criteria include:
1. Each set of inputs should only have one output. In our case, each material should only have one bandgap value.
2. There are no missing input or output values.

To tackle the first problem with duplicate data let's focus on the "Reliability" column to the far right.
Notice that for many of the chemical formulas in the dataset we have multiple measurements, with some being labeled as more reliable and others as less reliable. For now let's filter out all the data points (rows in the dataset) that don't have a Reliability of 1, the most reliable label and see if that removed our duplicate data.

this can be accomplished in the single line of code below.

</div> <!-- closes content -->

</div> <!-- closes lesson -->


In [10]:
# Filter for only Reliability 1
mastml_df_filtered = mastml_df[mastml_df["Reliability"]==1]

In [11]:
# Print filtered data
mastml_df_filtered

,index,chemicalFormula Clean,Band gap values Clean,Band gap units,Band gap method,Reliability
0,0,Li1F1,13.600,eV,Reflection,1
1,1,Li1F1,12.610,eV,Reflection,1
6,6,Li1Cl1,9.330,eV,Reflection,1
7,7,Li1Br1,7.950,eV,Absorption,1
9,9,Li3Sb1,1.000,eV,Thermal activation,1
...,...,...,...,...,...,...
1433,1445,Bi,0.015,eV,Magnetoreflection,1
1436,1448,Th1O2,5.750,eV,NaN,1
1437,1449,Th1O2,3.500,eV,Absorption,1
1443,1455,UO,1.500,eV,Thermal activation,1


Looking through the filtered data and paying attention to the chemical formula column there are still some formulas for which we have multiple measurements. Because we don't have another way to decide which data points to keep, let's average the values between these multiple measurements.  

To do this we'll use a method in Pandas (the dataframe package we are using to handle the data) called groupby which allows us to create groups of all of the identical formulas, and then average within each group.

In [12]:
mastml_df_clean = mastml_df_filtered.groupby("chemicalFormula Clean", as_index = False).mean()

mastml_df_clean

TypeError: Could not convert eVeV to numeric

<div class = "exercise">

### Exercise 1.2<a name="e1.2"></a>

Question:  
    1. Look at the starting dataframe mastml_df, How Many data points did we start with? (make sure to look at the very first dataframe we imported and not the one after filtering for reliability) 
    2. Now look at the cleaned dataframe mastml_df_clean, how many data points do we have now?  

[Check or Submit Answers](#a1.2)

</div>

<div class = "lesson">

### Evaluating Data availability<a name = "lesson1.3"></a>

<div class = "cellContent">

In general, the more data we have, the more confident we are at predicting properties of materials that are similar to our training data.

Now our dataset has been cleaned, let's start to look in more detail at the band gap values that we have. When preparing to work with a dataset, it's important to know what the values trying to be predicted look like. What's their scale and how are they distributed.

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
# generate basic statistics on our band gap values
mastml_df_clean["Band gap values Clean"].describe().round(3)

<div class = "exercise">

### Exercise 1.3<a name="e1.3"></a>

Question:  
    1. What is the range of band gap values?  
    2. Think ahead to once we build our model. Would a predicted error of 5 eV be considered small enough to be an accurate or useful prediction?  
    3. How about a predicted error of 0.5 eV?

[Check or Submit Answers](#a1.3)

</div>

Apart from just the ranges of values it is also useful to visualize the distribution of data.  
Let's build a simple histogram of the band gap values.

In [ ]:
# we'll also define a simple histogram plotting function to use later
def histogram_plot(data):
    fig1,ax1 = plt.subplots()
    ax1.hist(data,bins=range(13),density=1)
    ax1.set_xticks(range(13))
    ax1.set_xlabel('Measured Bandgap [eV]')
    ax1.set_ylabel('Counts [fraction]')
    plt.show()

histogram_plot(mastml_df_clean["Band gap values Clean"].astype("float"))

<div class = "exercise">

### Exercise 1.4<a name="e1.4"></a>

Questions:  
1. Is our band gap data balanced (i.e. uniformly distributed across its range)?
2. Given your answer to question 1, Would you expect that the model has similar performance between 0-2 eV as between 10-12 eV?

Challenges (optional questions):
3. What is one thing we can do to address this issue?

[Check or Submit Answers](#a1.4)

</div>

Let's also try to get a feel for the compositions present in our dataset.  
Specifically we'll focus on looking at which elements are present in the data, and in what quantity.

In [ ]:
# parse out individual elements for each formula using pymatgen's composition parser
element_list = list()
for idx in mastml_df.index:
    element_list.extend(pymatgen.core.composition.Composition(mastml_df["chemicalFormula Clean"][idx]).elements)

In [ ]:
# setup a counter to count each element
temp_counter = Counter(element_list)
element_tuples = list(zip(list(temp_counter.keys()),list(temp_counter.values())))
element_df = pd.DataFrame(element_tuples,columns=["Element","Count"])
element_df_sorted = element_df.sort_values(by=["Count"],ascending=False)

In [ ]:
element_df_sorted

<div class = "exercise">

### Exercise 1.5<a name="e1.5"></a>
Questions:  
1. What are the five most common elements in the dataset?  
2. What are the five least common elements in the dataset?  
3. Rank your confidence in the following predictions:  

    - predictions containing Oxygen (oxides)  
    - predictions containing Iridium  
    - predictions containing an element that doesn't appear in the dataset at all  

[Check or Submit Answers](#a1.5)

</div>

In [ ]:
# Output data to csv - note depending on when you run this the updated data file may have been pregenerated so this cell isn't technically necessary.
output_path = "../data/bandgap_data_v3.csv"

if os.path.isfile(output_path):
    print(output_path," exists, not creating new file")
else:
    mastml_df_clean.to_csv(output_path)

<div class = "section">

# 2. Feature Generation<a name="2"></a>
---
[Back to TOC](#toc)

<div class = "cellContent">

## Overview
This section covers generating features from chemical formulas in the dataset. This featurization step is necessary to turn the text representation of materials into a numerical representation that the models can understand! In order to make this notebook independent from outside code versioning as much as possible, we'll use pre-generated features using the MAST-ML code package instead of generating them in real time in this notebook. Though the features used are from a previous version of MAST-ML which generated a list of "composition average elemental properties" to represent each unique material.

## Learning Outcomes
1. Validate generated features by manual calculation
2. Assess generated features using criteria of variability
3. Inspect features for relative size and shape
3. Discuss qualities of a good feature


</div> <!-- closes content -->

</div> <!-- closes section -->

<div class = "lesson">

### MASTML Configuration<a name = "lesson2.1"></a>

<div class = "cellContent">

[Materials Simulation Toolkit for Machine Learning (MAST-ML)](https://mastmldocs.readthedocs.io/en/latest/) is an open-source Python package designed to broaden and accelerate the use of machine learning in materials science research. It supports a full end-to-end machine learning workflow including all of the steps that we code by hand in this lab, but here we will talk through how specifically it generates Composition Average Elemental Properties to featurize a dataset. 

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
# get a new dataframe of generated features from the pregenerated matml run.
cwd = os.getcwd()
generated_features_path = os.path.join(cwd,"./generated_features/generated_features.csv")
features_df = pd.read_csv(generated_features_path)

The raw MASTML output combines the original data and the generated features in one single dataframe, which isn't ideal. To make our next step (feature engineering) easier, We will split it into two dataframes:
1. `target_data_df`: target values (outputs)
2. `features_df`: features (inputs)

In [ ]:
#split features_df into two dataframes
target_data_df = pd.DataFrame([features_df["chemicalFormula Clean"],features_df["Band gap values Clean"],features_df["Band gap units"],features_df["index"],features_df["Reliability"]]).T
features_df = features_df.drop(columns=['index','Reliability','Band gap values Clean','Band gap units','chemicalFormula Clean'])

Now, let's take a look at our target values first. Note that it still contains other input information (such as chemical formula) to help you contextualize what the bandgap values mean. Later, we will drop these columns as they won't be used in the model training.

In [ ]:
target_data_df # our original dataset with inputs and outputs

Let's also take a look at the features generated. Looking at the column names you will notice that each of them follows the pattern of: ElementalProperty_composition_average

Some of these properties may be familiar to you such as AtomicWeight, which can be looked up in the periodic table of the elements. Others may be a bit harder to understand from their shorthand such as BCCefflatcnt, which stands for Body Centered Cubic effective lattice constant. In this case this property is describing information about how long certain bond lengths are within an idealized crystal of the element. Even though they are more complex they have still be tabulated by previous researchers and therefore MAST-ML is able to simply look them up from known resources to calculate the properties shown.

In [ ]:
features_df # features generated

<div class = "exercise">

### Exercise 2.1<a name="e2.1"></a>
Questions:  
1. How many features have we generated?  

[Check or Submit Answers](#a2.1)

</div>

<div class = "lesson">

### Validating compositional average features<a name = "lesson2.2"></a>

<div class = "cellContent">

The Features that we've generated are all compositional averages of elemental properties. Using an existing database of elemental properties, the MASTML code reads in each of the chemical formulas in the dataset and combines each elemental property for each element in the formula according to the following equation:  
  
\begin{equation}
Property\_CompositionAverage = \frac{A \cdot Property\_Value_A + B \cdot Property\_Value_B}{A + B}
\end{equation}
  
Where A and B are the amounts of each element in the formula.

</div> <!-- closes content -->

</div> <!-- closes lesson -->

<div class = "exercise">

### Exercise 2.2<a name="e2.2"></a>
Questions:  
1. Calculate the $AtomicNumber\_CompositionAverage$ for the chemical Formula $\text{Li}_3\text{Sb}_1. (AtomicNumber_\text{Li} = 3, AtomicNumber_\text{Sb} = 51)$

[Check or Submit Answers](#a2.2)

</div>

<div class = "section">

# 3. Feature Engineering<a name="3"></a>
---
[Back to TOC](#toc)

<div class = "cellContent">

## Overview
The next big step we need to do before building models and making prediction is to make sure our features are useful for modeling. We'll perform three checks on our features that should improve their usefulness. This step is similar to the initial dataset cleaning / pre-processing that we did at the start of the lab, except now we're focusing on "cleaning" our features instead of cleaning the output data and compositions.

Feature Engineering Steps:  
1) Remove Constant Columns  
2) Remove Highly Correlated Columns  
3) Normalize Features  


## Learning Outcomes
1. Explain benefits of feature normalization, removing correlated features, and removing constant features
2. Recall that good features should be able provide information about the target variable
3. Recall that good features provide unique information about the target variable
4. Define a basic normalization scheme
5. Execute a normalization scheme


</div> <!-- closes content -->

</div> <!-- closes section -->

<div class = "lesson">

### Remove constant columns<a name = "lesson3.1"></a>

<div class = "cellContent">

Columns that have a constant value for all data points have no information and can't possibly help the ML model learn anything.  
We'll throw those out.

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
# Remove Constant Columns
features_df_noconstant = features_df.loc[:, (features_df != features_df.iloc[0]).any()] 

# report number of columns
len(features_df_noconstant.columns)

<div class = "exercise">

### Exercise 3.1<a name="e3.1"></a>
Questions:  
1. How many features do we have left?  
2. Should you worry about having too few useful features? 

[Check or Submit Answers](#a3.1)

</div>

<div class = "lesson">

### remove highly correlated features<a name = "lesson3.2"></a>

<div class = "cellContent">

Features that are extremely similar don't give any additional information beyond the first appearance.  
They can also confuse the model by giving very similar information that may essentially overload the model.  
Before we remove features, let's calculate the `correlation matrix` for all the features. 

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
# Remove Highly correlated Features
# using notes here for methodology: https://chrisalbon.com/machine_learning/feature_selection/drop_highly_correlated_features/

features_corr_df = features_df_noconstant.corr(method="pearson").abs()
features_corr_df.iloc[:5, :5] # Preview the first 5 rows/columns of the correlation matrix

A better way to interpret this correlation matrix is by plotting a heatmap: The darker the color on the plot, the more highly correlated features are.   
Note the diagonal line with all 1 values. This is because each feature is by definition perfectly correlated with itself.

In [ ]:
# before removing correlated features
fig1, ax1 = plt.subplots(figsize=(10,5))
c = ax1.pcolor(features_corr_df,cmap="Blues")
ax1.set_ylim(ax1.get_ylim()[::-1])
ax1.xaxis.set_ticks_position('top')
ax1.xaxis.set_label_position('top')
ax1.set_xlabel('Feature Numbers')
ax1.set_ylabel('Feature Numbers')
ax1.set_aspect('equal')
plt.colorbar(c,ax=ax1)
plt.show()

In [ ]:
# Filter the features with correlation coefficients above 0.95
upper = features_corr_df.where(np.triu(np.ones(features_corr_df.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
features_df_lowcorr = features_df_noconstant.drop(columns=to_drop)
# recalculate the correlation matrix so we can compare
features_corr_df_update = features_df_lowcorr.corr(method="pearson").abs()

In [ ]:
# plot correlation after removing highly correlated features

fig1, (ax1,ax2) = plt.subplots(1,2,figsize=(15,5))
c1 = ax1.pcolor(features_corr_df,cmap="Blues")
ax1.set_ylim(ax1.get_ylim()[::-1])
ax1.xaxis.set_ticks_position('top')
ax1.xaxis.set_label_position('top')
ax1.set_xlabel('Feature Numbers')
ax1.set_ylabel('Feature Numbers')
ax1.set_aspect('equal')

plt.colorbar(c1,ax=ax1)

c2 = ax2.pcolor(features_corr_df_update,cmap="Blues")
ax2.set_ylim(ax2.get_ylim()[::-1])
ax2.xaxis.set_ticks_position('top')
ax2.xaxis.set_label_position('top')
ax2.set_xlabel('Feature Numbers')
ax2.set_ylabel('Feature Numbers')
ax2.set_aspect('equal')
plt.colorbar(c2,ax=ax2)
plt.show()

In [ ]:
len(features_df_lowcorr.columns)

<div class = "exercise">

### Exercise 3.2<a name="e3.2"></a>
Questions:  
1. After filtering for highly correlated features how many features do we have left?  
2. Are we worried about having too few useful features? 

[Check or Submit Answers](#a3.2)

</div>

<div class = "lesson">

### feature normalization<a name = "lesson3.3"></a>

<div class = "cellContent">

Finally the last thing that we need to do is perform some normalization or rescaling of our features.  
Lets take a look at the updated feature dataset and pay attention to the types of features, their ranges, and scale similar to what we did for the band gap earlier

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
features_df_lowcorr

It should be fairly apparent that our features come in many shapes and sizes. Machine Learning algorithms can be very sensitive to these differences.  
For example one feature may be several orders of magnitude larger in values and in range of values.  
This can make some algorithms significantly biased towards those features so the best practice is usually to perform some alteration to make all the features look similar, while still preserving the information they contain

In our case we're going to linearly rescale the features so that they all have the same minimum and same maximum. If you're interested in checking of the details of how this is done you can check out the documentation for the Scikit-learn method we'll be using: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [ ]:
minmax_features = MinMaxScaler().fit_transform(features_df_lowcorr)
minmax_features_df = pd.DataFrame(minmax_features,columns=features_df_lowcorr.columns)
minmax_features_df.iloc[:5, :5]

<font color=blue> challenge: Turn off the feature normalization. Feature normalization is a common practice to enable models to better learn from multiple features when some are on significantly different scales. 
Try removing this section to see how the later results are affected. In the case of the decision tree / random forest model being used by default this may not be the case, but what about other model types? Try doing the same thing with a Kernel Ridge Regression Model for example. </font>

Notice how compared to some of the previous sections, performing the scaling only took a few lines of code. This is the power of using existing code packages and tools that are already out there!

<div class = "section">

# 4. Setup for Model Evaluation<a name="4"></a>
---
[Back to TOC](#toc)

<div class = "cellContent">

## Overview
Before jumping in to model building, we have one last thing to think about: How are we going to know how well our models are performing?  

We can test the model by asking it to predict the bandgap value of materials, compare it with measured values, and calculate its prediction error using a range of error metrics (e.g. RMSE, R2, etc), but the naive approach of training and testing the model using the same dataset gives us a <u>biased estimate</u> of model error because the model has *seen* all the data we are asking it to predict already.  

To get an <u>unbiased estimate</u> of model error, we can employ a simple <u>cross-validation</u> technique: Setting aside a subset of data as our <u>test set</u>, which the model won't access during the training process. The rest of the data constitutes the <u>training set</u>, which is used to train the model. This allows us to then evaluate model performance by comparing model prediction on both seen data (training set) and unseen data (test set).

## Learning Outcomes
1. Explain how model evaluation is tied directly to ML application
2. Propose a train test split to only predict high bandgap materials
3. Define how training and test splits are used by a model


</div> <!-- closes content -->

</div> <!-- closes section -->

<div class = "lesson">

### Establishing train/test split<a name = "lesson4.1"></a>

<div class = "cellContent">
    
For now lets split off 10% of the data for testing.

</div> <!-- closes content -->

</div> <!-- closes lesson -->

First, we store our cleaned and normalized inputs and outputs in new variables `X` and `y` for easier understanding and manipulation.

In [ ]:
X = minmax_features_df                         # inputs/features 
y = target_data_df["Band gap values Clean"] # outputs/targets

Normally, if you are using machine learning to predict the bandgap (or other properties) of a novel material, you won't know its real bandgap until you fabricate and measure it in the lab, which is bad news for instructors: What's the point if we weren't able to validate the predictions and show you the power of machine learning?   

Therefore for instructional purposes, we will stage our prediction by using 5 common materials with known bandgap values instead - Si, SiO2, C, NaCl, and Sn - and removing them from the dataset.

The following code accomplishes the above and is not important otherwise. Note that you do **NOT** need this step in a real research setting.

In [ ]:
# Find prediction compounds and generate inputs for them to make predictions later.
def extract_predictions(formula="string"):
    index_prediction = target_data_df[target_data_df["chemicalFormula Clean"]==formula].index
    xpredict = X.loc[index_prediction].copy()
    ypredict = y.loc[index_prediction]
    
    return (index_prediction,xpredict,ypredict)

In [ ]:
index_predict_Si, xpredict_Si, ypredict_Si = extract_predictions(formula="Si")
index_predict_SiO2, xpredict_SiO2, ypredict_SiO2 = extract_predictions(formula="Si1O2")
index_predict_C, xpredict_C, ypredict_C = extract_predictions(formula="C")
index_predict_Sn, xpredict_Sn, ypredict_Sn = extract_predictions(formula="Sn")
index_predict_NaCl, xpredict_NaCl, ypredict_NaCl = extract_predictions(formula="Na1Cl1")

X_predict = X.drop(index=index_predict_Si.to_list()+index_predict_SiO2.to_list()+index_predict_C.to_list()+index_predict_Sn.to_list()+index_predict_NaCl.to_list())
y_predict = y.drop(index=index_predict_Si.to_list()+index_predict_SiO2.to_list()+index_predict_C.to_list()+index_predict_Sn.to_list()+index_predict_NaCl.to_list())

Then, we use the `train_test_split()` method from the `scikit-learn` package to generate the split. In this case, our input data `X` and output data `y` are split into 4 parts:    
- `X_train`: training set input data  
- `X_test`: test set input data  
- `y_train`: training set output data  
- `y_test`: test set output data  

We will continue referencing these 4 objects throughout the rest of this lab.

In [ ]:
# Generate train/test split by reserving 10% of data as test set

test_fraction = 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_fraction, shuffle=True,random_state=seed)

<div class = "lesson">

### Evaluating train/test split<a name = "lesson4.2"></a>

<div class = "cellContent">
    
One thing that can be interesting to check is how "representative" the splits are of the full dataset. In an ideal world we'd have enough data that even splitting out 10% of it would have enough data in each set to accurately reproduce the whole dataset.  

Below lets plot the same histogram of band gap values as we did before, and then plot the same for the train and test set.

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(10,5), sharex = True, gridspec_kw={'hspace': 0})
fig.set_tight_layout(False)
myarray = mastml_df_clean["Band gap values Clean"]

bins = np.true_divide(range(28),2)

l1 = sns.distplot(y_train.astype("float"), hist = True, norm_hist = True, kde = False, bins = bins, hist_kws={"edgecolor": "white"}, label = 'training set', ax = ax1)
l2 = sns.distplot(y_test.astype("float"), hist = True, norm_hist = True, kde = False, bins = bins, hist_kws={"edgecolor": "white", "color": "orange"}, label = 'test set', ax = ax2)
l3 = sns.distplot(myarray, hist = True, norm_hist = True, kde = False, bins = bins, hist_kws={"histtype": "step","linewidth": 3, "alpha": 1, "color": "grey"}, ax = ax1)
l4 = sns.distplot(myarray, hist = True, norm_hist = True, kde = False, bins = bins, hist_kws={"histtype": "step","linewidth": 3, "alpha": 1, "color": "grey"}, label = 'full dataset', ax = ax2)


ax1.set_xticks(range(14))
ax2.set_xticks(range(14))
ax2.xaxis.label.set_visible(False)
handles, labels = [(a + b) for a, b in zip(ax1.get_legend_handles_labels(), ax2.get_legend_handles_labels())]
fig.suptitle('Comparing histograms of the train/test split')
fig.add_subplot(111, frame_on=False)
plt.tick_params(labelcolor="none", bottom=False, left=False)
plt.legend(handles, labels, loc = 'center left', bbox_to_anchor=(1, 0.5),prop={'size': 16})
plt.xlabel('Measured Bandgap (eV)')
_ = plt.ylabel('Density')

<div class = "exercise">

### Exercise 4.1<a name="e4.1"></a>
Questions:  
1. Does it look like we have enough data that the test split is identical to the full dataset and the train split?

[Check Answers](#a4.1)

</div>

Answers:  
1.  

<div class = "section">

# 5. Fitting and Evaluating a Default Model<a name="5"></a>
---
[Back to TOC](#toc)

<div class = "cellContent">

## Overview

Significance:
Status quo: what did we do in last section
    So far, our dataset has been inspected, properly cleaned and split into training and test sets.
Gap: 
Fill gap: 

what you will do: Now we finally get to the fun part: Building models and make predictions! We're going to fit a decision tree model to the training data we created in the previous section. Then we'll use that model to predict both the test data and the training data.


## Learning Outcomes
By the end of this section, you will be able to:
1. Evaluate model performance
    1. Analyze parity plots for qualitative performance of models
    2. Interpret error metrics to compare model performance
2. Recognize value of test data predictions in assessing predictive ability compared to training data
3. Identify key indicators of overfitting
4. Estimate model performance for two different ML applications


</div> <!-- closes content -->

</div> <!-- closes section -->

<div class = "lesson">

### Fitting the decision tree model<a name = "lesson5.1"></a>

<div class = "cellContent">

We'll train or "fit" a decision tree model using the `RandomForestRegressor().fit()` class from the `scikit-learn` package we imported at the beginning of the notebook. Notice how this only takes 1 single line of code because we're leveraging existing code package that's been built for us already. Also note that the actual class name that we're using is called RandomForestRegressor. This RandomForestRegressor class can be configured to be identical to a decision tree, however the reason we'll use it is because later on it will give us more flexibility when we start changing the model. A decision tree model is basically a random forest model with 1 tree, so we'll set the `n_estimators` hyperparameter to 1 to make the model mimic a single decision tree.

Keep in mind that machine learning is much more than this 1 line of code: You have already gone through a ton of preprocessing and decision making to reach this step!

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
Default_model = RandomForestRegressor(random_state=seed,n_estimators=1,bootstrap=False).fit(X_train,y_train) # fit the decision tree model
print('Model training complete.')
# print('Tree depth:', [estimator.tree_.max_depth for estimator in Default_model.estimators_])
# for importance in zip (estimator.feature_importances_ for estimator in Default_model.estimators_):
#     print (importance)
#print('Leaf nodes:',[estimator.tree_.n_leaves for estimator in Default_model.estimators_])

The outputs above describes the <u>hyperparameters</u> selected (in this case, by default) to fit the decision tree model.  and the <u>parameters</u> being generated in the training process. You may also wonder what the decision tree *looks* like, and we will visualize the entire tree later.  

We'll also go into more detail about what these (hyper)parameters are at a later time when they become more relevant. For now, we're glossing over because simply knowing these (hyper)parameters doesn't help us evaluate model quality until we have seen its performance: How accurately and precisely can our decision tree model predict bandgaps? We will jump into that.

As one last motivation as we start asssessing our model, lets predict two band gaps of materials you're probably familiar with, Silicon and Silica. Silicon is used in practically every electronic device as a semiconductor, and Silica is basic window glass. You can look up the values of their band gaps fro reference, but look how just in a few lines of code the model can already give us a rough idea of their values. We know Silicon is a semi-conductor and it's bandgap should be fairly low, while the band gap for Silica has to be much higher because window glass shouldn't absorb any light at all. Based on these predictions it seems like the model can already pick up on these trends! But as we've been mentioning, just making a few select predictions is not a good way to measure overall performance, in the next sections we'll dig into more robus ways to measure the performance!

In [ ]:
Default_model_all_data = RandomForestRegressor(random_state=seed,n_estimators=1,bootstrap=False).fit(X_predict,y_predict)

print("Predicting Silicon Band Gap: ",Default_model_all_data.predict(xpredict_Si))

print("Predicting Silica Band Gap: ",Default_model_all_data.predict(xpredict_SiO2))

<div class = "lesson">

### Evaluating model performance on training and test data<a name = "lesson5.2"></a>

<div class = "cellContent">

To evaluate the model performance, we will use it to predict bandgaps of materials it was trained on (training set) as well as materials it has not seen before (test set), and we will compare its performance on both datasets.

There are two main ways to understand and evaluate prediction performance:
1. Qualitatively, we can create a scatter plot with predicted values on the y-axis and the actual measured values on the x-axis, known as <u>parity plots</u> or <u>predicted versus actual plots</u>. The more closely aligned the data points are to the ideal diagonal line, where each prediction matches the measured value perfectly, the better the model performs on that specific dataset. Parity plots also surface outliers, trends and patterns in the prediction results.
2. Quantitatively, we can measure model performance using statistical error metrics. Some of these error metrics, such as RMSE and R2, will be familiar to you if you have taken an introductory statistics class.

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
Train_predictions = Default_model.predict(X_train)    # Make predictions on training data
Test_predictions = Default_model.predict(X_test)      # Make predictions on testing data

parity_plots_side_by_side(y_train,Train_predictions,y_test,Test_predictions,title_left="Training Data Parity Plot",title_right="Test Data Parity Plot") # build both plots
parity_stats_side_by_side(y_train,Train_predictions,y_test,Test_predictions,"Training Data","Test Data")  # print error metrics for training data

We've generated a few different error metrics which we can use to asses the model's performance. One that we'll focus on throughout the lab is the Root Mean Squared Error (RMSE), which we are going to use as a rough error bar when talking about predictive ability of the model. Meaning when we're analyzing performance and asking questions about how accurate the model is is making predictions this is the statistic we'll reference. It's important to note that this is just one choice we could make for assigning an error bar to the model's predictions. There are other, more complex methods for generating error bars for model predictions, and we're going to ignore those for now in favor of simplicity. So whenever we're asking you to think about the predictive power of the model for now think of model predictions as having a predicted value plus or minus the RMSE.

<div class = "exercise">

### Exercise 5.1<a name="e5.1"></a>
Questions:  
1. Take a look at the training parity plot. Evaluate model prediction on training data by answering the following:  
    1. Is there enough information in the features to make predictions? Do the features model this data (training) well?
    2. Are there any outliers?  
    3. Does it consistently overpredict/underpredict bandgap values in any particular range?  

Take a look at all the training data statistics (RMSE, RMSE/std, MAE, R2). Assume these metrics, specifically RMSE, can be used to put an error bar on any predictions we make. Recall that we identified two ML applications for this model, and answer:

2. Can we use this model to predict bandgap values of materials for making single-junction solar cell, which requires a bandgap between 1.1 and 1.7eV?
3. Can we use this model to predict high bandgap materials above 3 eV?

[Check or Submit Answers](#a5.1)

</div>

<div class = "exercise">

### Exercise 5.2<a name="e5.2"></a>
Questions:  
1. Take a look at the test parity plot. Evaluate model prediction on test data by answering the following:  
    1. Is there enough information in the features to make predictions?
    2. Are there any outliers?  
    3. Does it consistently overpredict/underpredict bandgap values in any particular range? 
2. Take a look at all the test data statistics (RMSE, RMSE/std, MAE, R2). Given what you now know about model performance on test data, can we still use this model to predict bandgap values of materials for making single-junction solar cell, which requires a bandgap between 1.1 and 1.7eV?  
3. Again think about the use case for identifying high band gap materials above 3 eV, could the model differentiate between low and high bandgap materials?

[Check or Submit Answers](#a5.2)

</div>

<div class = "exercise">

### Exercise 5.3<a name="e5.3"></a>
Questions:  
1. Compare both the parity plots and performance statistics for the training and test set. Is the model performing better on one set than the other, or is there no difference? (No calculation needed.)  
2. Which of the following most accurately describes this model: Underfit, overfit, or neither?
3. Should we use training data or test data to estimate model prediction performance?


[Check or Submit Answers](#a5.3)

</div>

<div class = "lesson">

### Visualizing and interpreting the decision tree model (OPTIONAL)<a name = "lesson5.3"></a>

<div class = "cellContent">

It's always important and helpful to understand how your ML model makes predictions - even more so for materials scientists, who are more likely to trust the model prediction and fabricate the recommended materials if the prediction has an explanation that matches their physical intuition (or defies it in a scientifically justifiable way).

However, it's often a challenge because many ML models are "**black boxes**", the inner workings of which are not interpretable to humans at all. Decision trees are chosen for this lab partly because they are more interpretable than many other models. Even so, it becomes harder to extract physical instinct from them as they grow larger.

As a final step, we will visualize the default decision tree model to help you understand what we've built. While the visualization doesn't quantify its performance, it illustrates which features are important and explains the difference between model performances on training and test data.

</div> <!-- closes content -->

</div> <!-- closes lesson -->

<div class = "info">

### *Large cell output!*


<div class = "cellContent">
    
The output of the next cell is very large! Click the left side of the output to expand it for a full view, or click again to minimize it.  

Minimizing the output after viewing also makes it easier to interact with the rest of the notebook.

</div> <!-- closes content -->
</div> <!-- closes protip -->

In [ ]:
# generate an image of the default decision tree
dot_data = sklearn.tree.export_graphviz(Default_model.estimators_[0],out_file=None,feature_names=features_df_lowcorr.columns,filled=True,rounded=True,special_characters=True)
graph = graphviz.Source(dot_data)
graph

<div class = "info">

<div class = "cellContent">
    
Locate the decision node `Density_composition_average ≤ 0.059` in the visualization above using Ctrl+F (Cmd+F on Macs).

</div> <!-- closes content -->
</div> <!-- closes protip -->


This visualization explicitly constructs each node in the default tree. Using the decision node `Density_composition_average ≤ 0.059` as an example, 

 It lists which feature the node splits on, gives the mse for the data at that split, how many samples are at the node, and the value of the estimated band gap if it was a leaf node.  
It should be immediately apparent that the tree is incredibly large, and and so we'll pick a few things to focus on as we look through it briefly.

<div class = "exercise">

### Exercise 5.4<a name="e5.4"></a>
Questions:  
1. Looking at the Density_composition_average node identified above, how many samples from the training data are included at this node?  
2. Trace the branch down from this node until you find a leaf node. How many samples are included at this leaf node?  
3. Inspect a few other leaf nodes until you identify how many samples are at the majority of leaf nodes, how many samples is this?  
4. based on your previous analysis of overfitting, underfitting, or well fit does this support that previous claim about how the model is fit?  


[Check or Submit Answers](#a5.4)

</div>

<font color=blue> challenge (optional): We're currently leaving out a fairly small percentage of the data, 10%. Try going back and changing this to a few different values to see how it changes the results. For example try: 5%, 25%, 50%, 75%.  
To do this you can edit the test_fraction parameter at the start of Section 4 and rerun the cells after that.</font>

<div class = "section">

# 6. Improving the Model by Optimizing Hyperparameters<a name="6"></a>
---
[Back to TOC](#toc)

<div class = "cellContent">

## Overview (edited)

In the previous section, we evaluated the performance of a default model based on its performance in predicting a single set of test data. While this model predicted the training data almost perfectly, predictions of the testing data showed significantly worse performance. This combination of a model that perfectly reproduces training data, but can't predict data it hasn't seen is a classic sign of <u>overfitting</u>. One way to address this issue to to try to alter the hyperparameters of the model.

When building our first model we used default settings from the `scikit-learn` package, we can attempt to improve the performance by optimizing the <u>hyperparameters</u> of a model (think of them as *tunable knobs*). 

This will be the penultimate task in this lab, before we select a final model and make predictions.

Remember that a machine learning model has two types of parameters:  
 - The first are the regular <u>parameters</u> of the model. These would be the coefficients in the case of a linear model; in the case of the decision tree, they are the nodes that get generated when the model is fit, and we don't set them directly.  
 - The second are the <u>hyperparameters</u>. These are the variables that we adjust to change how the model learns and how it gets fit. For example, for decision trees, we can limit how complex and large the tree can be by setting a number of hyperparameters including the maximum tree depth and the maximum leaf nodes. We can also increase the number of trees that we train and make a model that is an average prediction of these multiple trees.  

In this lab, we'll focus in on one hyperparameter specifically called <i>n\_estimators</i> . Each estimator is one decision tree, and by using the RandomForestRegressor class from sci-kit-learn we can increase this number to build multiple individual trees which will all try to predict our bandgaps. Taking average results from multiple decision trees can often improve model performance and reduces overfitting that can occur in a single decision tree.



## Learning Outcomes
1. Identify a hyperparameter for the random forest model
2. Perform a series of grid searches to find an optimal hyperparameter
3. Evaluate model performance for different hyperparameter configurations
4. Choose an optimal hyperparameter configuration
5. Assess overfitting from learning curves
6. Recall final optimized model training step before predictions can be made
7. Compare model validation techniques: k-fold CV vs. train/test split
8. Evaluate model performance for two different applications
9. Recall decision tree (as evidence from inspections) is in high variance regime
10. Recall random forest allows smoothing/averaging across many trees to reduce variance


</div> <!-- closes content -->

</div> <!-- closes section -->

<div class = "lesson">

### Defining a parameter space<a name = "lesson6.2"></a>

<div class = "cellContent">

To optimize the hyperparameters we will perform a series of "grid searches". As the name implies we will define a grid of hyperparameter values to try, and then build models at each of those grid points. By assessing model performance at each grid point we can make a plot to see how model performance changes as a function of the hyperparameters. 

As mentioned above we'll be focusing on just varying one hyperparameter, but fundamentally we could vary any number of them at a time and make a higher dimensional grid that explores them all at once. The fundamental contraint to this type of strategy is the amount of time we are willing to wait for results. The more grid point we define, the more models we need to build, and therefore the more time it will take for the code to complete.

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
print('Default model uses the following hyperparameters:\n') # print default hyperparameters used
pprint(Default_model.get_params())

In [ ]:
# set up hyperparameter grid (a dictionary of hyperparameter candidates that we want the optimization strategy to consider)

# EDIT LIST TO TRY DIFFERENT VALUES!
### MAKE EDITS BELOW HERE ###

number_of_trees = [1,10,25,50]

### MAKE EDITS ABOVE HERE ###

opt_dict = {'n_estimators':number_of_trees,'bootstrap':['True']}

<div class = "lesson">

### Setting up a cross-validation scheme<a name = "lesson6.1"></a>

<div class = "cellContent">

Before we setup our optimization we'll want to readress our methodology for model assessment. Previously we made a single Train vs Test split in the data. This type of assessment is nice as a final check of a model once the hyperparameters are set, however think about what would happen if we only used that single Test split as our method for assessment. The we could find the best model at predicting that specific subset of data, but not the model that is best at predicting all of our available data. To adress this we'll use a cross-validation strategy from scikit-learn called `RepeatedKFold` cross-validation. The main different is that instead of making a single split in the data we will make 'K' splits in the data and predict each of those in turn by training on the remaining data. This means every data point is used in both training and testing. The "Repeated" addition to the KFold method means that the process is repeated a number of times so that there are a total of "K" x "Repeats" splits in the model. Now if we minimze the average error in predicting each split individually we can hopefully find a model that performns well in predicting all of our data!


We'll set the number of splits to be 5, and also set a number of repeats to perform as 5. This means that in total we have 25 splits being generated in the training data.  

Note: We're still leaving out the previously established testing data and not including it here. This is so we can go back after optimization and see how we do at predicting that data!

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
# kfold = KFold(n_splits=5,random_state=seed,shuffle=True)
kfold = RepeatedKFold(n_splits=5,
                      random_state=seed,
                      n_repeats=5)

<div class = "lesson">

### Setting up a grid search<a name = "lesson6.3"></a>

<div class = "cellContent">

With the cross validation established, and a grid of hyperparameters assigned we just need to put everything together and build all combinations of models with the various splits and hyperparameters. Conveniently `scikit-learn` has build a method for doing just that.

The `scikit-learn` documentation on hyperparameter tuning describes such a grid search as consisting of:

1. an estimator (our decision tree regressor, for example);
2. a parameter space;
3. a method for searching or sampling candidates (we are limiting ourselves to a simple grid search in this lab);
4. a cross-validation scheme; and
5. a score function.

*Note: While we are only demonstrating the simplest case of hyperparameter tuning in this lab, you should feel comfortable reading the scikit-learn documentation for continued learning after completing this lab: https://scikit-learn.org/stable/modules/grid_search.html*

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
# define a grid search strategy

import time

CV = GridSearchCV(Default_model, # 1. the model whose hyperparamter is being optimized right now
                  opt_dict,   # 2. a dictionary of values that we want the grid search to use
                  cv=kfold,   # 4. k-fold cross-validation strategy is used to define training and validation splits (note this is separate from test splits) to be used for each grid point
                  return_train_score=True,
                  scoring=['neg_mean_squared_error','r2','neg_mean_absolute_error'], # 5. the performance metrics to be reported at each grid point specified in opt_dict
                  refit='neg_mean_squared_error')

# perform grid search
tic = time.perf_counter() # start timer

CV = CV.fit(X_train,y_train)

toc = time.perf_counter() # stop timer

# print results
print(f"Grid search completed in {toc - tic:0.3f} seconds.")
print(CV.best_params_)

<div class = "info">

### *Why do some score functions have a "neg_" prefix?*

<div class = "cellContent">

Some of the score functions below have "neg" or negative in front. That's just done for computational reasons and doesn't change the actual metrics. We'll invert them when analyzing later. 


</div> <!-- closes content -->

</div> <!-- closes protip -->

And just like that we've performed the grid search! to visualize the results see the code blocks below.

<div class = "lesson">

### Visualizing the learning curve<a name = "lesson6.4"></a>

<div class = "cellContent">

How do we find out the best value for the hyperparameter `n_estimators` from the grid search?  
Let's check how the performance changed as a function of `n_estimators` by plotting the average Mean Squared Error (MSE) for both the training and validation splits for each hyperparameter choice (specified in `opt_dict`).

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
# plot number of trees vs train and test MSE

opt_dict_array = opt_dict["n_estimators"]                     # array of grid points (x-axis)
train_mse = CV.cv_results_["mean_train_neg_mean_squared_error"] # MSE of training set at each grid point (y-axis)
test_mse = CV.cv_results_["mean_test_neg_mean_squared_error"]   # MSE of test set at each grid point (y-axis)

fig1,ax1 = plt.subplots(figsize=(8,4))
ax1.scatter(opt_dict_array, -train_mse)
ax1.scatter(opt_dict_array, -test_mse)
# ax1.fill_between(opt_dict_array, -train_mse, -test_mse, alpha=0.1)
ax1.set_xlabel('Number of Trees')
ax1.set_ylabel('Mean Squared Error')
ax1.set_title('Compare training and validation MSE vs hyperparameter')
plt.legend(["Training","Validation","difference"])
plt.show()
print("Minimum Mean Squared Error: ", round(min(-test_mse),4))
print("Number of Trees at minimum: ", opt_dict_array[np.argmin(-test_mse)])

<div class = "exercise">

### Exercise 6.1<a name="e6.1"></a>
Before diving into the questions lets run a few different grid searches to get a feel for how it works. By default the notebook is setup with a very rough grid search which should run quickly. be careful adding too many grid points because it is possible to slow down the grid search to the point of not finishing in hours or days. As a rule of thumb lets not set number of trees to be above 100, and lets not include more than 10 individual grid points in any one search. Try to get a sense of how performance varies with number of trees. Before answering questions below set your grid to be [1,3,5,7,10,15,20,50] which should give a reasonable spread of values. As a reminder these edits are made in the section above "defining a parameter space" 

Questions:  
1. Looking just at the training data curve above, what is the number for `n_estimators` that gives the smallest MSE?
2. How about for the validation data? Which value for `n_estimators` gives the smallest MSE?  
3. Look at the trend in validation data. If n_estimators was increased beyond 50 do you think the model would change in performance significantly?  
4. Do we expect the model get more complex as we increase the number of trees?  
5. Looking at just the validation curve, does increasing number of trees seem to increase overfitting at any point on the curve? This would show up as worsening performance in the validation data as trees increase.

[Check or Submit Answers](#a6.1)

</div>

<div class = "lesson">

### Default vs. optimized model: cross-validation performance<a name = "lesson6.5"></a>

<div class = "cellContent">

Looking at the previous learning cuve of performance versus the number of trees we can see the more trees seems to be giving better performance, however past 20 trees the rate of improvement seems to decrease dramatically. Going forward we'll assume we're using a forest of 50 trees which should give significant improvement in performance compared to where we started.

Note: Recall that at each grid point, we are splitting the dataset into 5 random folds and repeating this process 5 times, which means we have trained 50 different decision tree models using the same hyperparameter at that grid point total. Every error metric you see below is the average value across 25 different models with the same hyperparameters but different splits of the training data.

Error metrics below are for models with the best hyperparameter from the previous grid search, so make sure the last time you ran the grid search the largest number of trees was 50! You can check if you did this correctly with the cell block below, the `n_estimators` value should be 50

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
# check what the best parameters identified in the grid search were
CV.best_params_

In [ ]:
# Extract cross validation performance metrics for the optimized model
opt_CV_stats = CV_best_stats(CV,y_train)

To compare back to our default model we can construct another grid search that only uses "1" for `n_estimators`. That way it will still be the best model available.

We do CV on a single grid point ("1"), build 25 different models, and average across them to get the results below.

Now we can directly compare the model's performance on these metrics generated from the Kfold cross validation.

In [ ]:
default_opt_dict = {'n_estimators':[1]}

default_CV = GridSearchCV(Default_model,
                          default_opt_dict,
                          cv=kfold,
                          return_train_score=True,
                          scoring=['neg_mean_squared_error','r2','neg_mean_absolute_error'],
                          refit='neg_mean_squared_error')
default_CV = default_CV.fit(X_train,y_train)

default_CV_stats = CV_best_stats(default_CV,y_train)

<div class = "exercise">

### Exercise 6.2<a name="e6.2"></a>
Questions:  
1. Do we get improvement in the RMSE between the default and optimized model? What is the percentage improvement ($|RMSE_{test opt}-RMSE_{test default}|/RMSE_{test default}$)?   

For this lab we'll assume that the RMSE metric gives a reasonable error estimate on predictions. So any prediction we made we'll assume it has + or - the CV RMSE value.   
2. Assuming this level of accuraccy from the optimized model. Is our model accurate enough to predict single-junction solar materials? where the key design metric is having a band gap between 1.1 eV and 1.7 eV?  
3. How about our other task. Is the optimized model accurate enough to predict high bandgap materials? where the key metric is ensuring predictions are above 3 eV?

[Check or Submit Answers](#a6.2)

</div>

<div class = "lesson">

### Default vs. optimized model: training and test data performance<a name = "lesson6.6"></a>

<div class = "cellContent">

The goal is to optimize performance on unseen data

Now, remember that we also have the set of Test data that we held out before any hyperparameter optimization. Lets see how the model has changed in predicting that data. We'll make the same two parity plots as before where we use the new optimized hyperparameters.

We use the optimized hyperparameters to refit the model using the entire training set.

</div> <!-- closes content -->

</div> <!-- closes lesson -->

In [ ]:
# Refit the model using the best hyperparameters
DT2 = CV.best_estimator_.fit(X_train,y_train)

# predict both the train and test data
Train_predictions2 = DT2.predict(X_train)
Test_predictions2 = DT2.predict(X_test)

In [ ]:
parity_plots_side_by_side(y_train,Train_predictions,y_train,Train_predictions2,title_left="Training Parity Plot (Default Model)",title_right="Training Parity Plot (Optimized Model)") # build both plots
parity_stats_side_by_side(y_train,Train_predictions,y_train,Train_predictions2,"Training Set (Default Model)","Training Set (Optimized Model)")

<div class = "exercise">

### Exercise 6.3<a name="e6.3"></a>
Questions:  
1. Look at both the parity plots and the training data statistics. Does the optimized model do better or worse at predicting the training data than the default model?  
2. Do prediction performances on training data give you enough information to decide which model is more likely to give better predictions on Si and $\text{SiO}_2$, which are not in the training set or the test set? Another way to ask this is does the training data result tell us anything about the predictive power of the model?

[Check or Submit Answers](#a6.3)

</div>

In [ ]:
parity_plots_side_by_side(y_test,Test_predictions,y_test,Test_predictions2, title_left="Test Parity Plot (Default Model)",title_right="Test Parity Plot (Optimized Model)") # build both plots
parity_stats_side_by_side(y_test,Test_predictions,y_test,Test_predictions2,"Test Set (Default Model)","Test Set (Optimized Model)")

<div class = "exercise">

### Exercise 6.4<a name="e6.4"></a>
Questions:  
1. Just looking at the testing data statistics, does the optimized model do better or worse at predicting the testing data?
2. Compare the difference between train and test RMSE for the default and optimized model. Did the difference between training and test performance increase or decrease after hyperparameter optimization?  
3. Is this evidence that the optimized model is more overfit or less overfit?  
[Check or Submit Answers](#a6.4)

</div>

<div class = "exercise">

### Exercise 6.5<a name="e6.5"></a>
Questions:  

1. Based on your answers to exercise 6.3 and 6.4, if you were to choose between the default model and the optimized model, what decision will you make? Give at least 3 criteria for your evaluation.

[Check or Submit Answers](#a6.5)

</div>

<div class = "section">

# 7. Making Predictions<a name="7"></a>
---
[Back to TOC](#toc)

<div class = "cellContent">

## Overview
In the final section of the lab we'll do one final test to gauge the model's performance. We've been trying throughout the lab to highlight the importance of keeping a model's use case in mind throughout the process and building and training. In our case we are focusing on two potential use cases. In predicting solar materials which require a bandgap between 1.1 and 1.7 eV, and wide bandgap materials which want a bandgap above 3 eV. 

In the more general sense, important questions to ask are things like:  
1) what is the necessary accuracy for predictions I want to make? For example we've highlighted two use cases for our band gap model, one that requires a much more accurate model than the other.  
2) What is the feature space I want to predict in (What compositions are we interested in predicting)? Our dataset is trained entirely on binary and elementary semiconductors. Making predictions outside of this space would be very risky as a lot of our error estimates might break down.
 

## Learning Outcomes
1. Judge model performance for two different applications
2. Assess previous error predictions
3. Propose improvements to an existing model


</div> <!-- closes content -->

</div> <!-- closes section -->

Remember back when we first trained the model and predicted Silicon and Silica? Let do the same thing for fun with the optimized model. The values have likely shifted.

When we fit the DT3 model we use the X_predict and y_predict versions of the dataset in which we removed 5 compounds so that we could predict them now. Note that these predictions are a bit artificial because when we did the model optimization this data was included. In a true research environment this isn't something you'd want to do.

In [ ]:
# fit model to all data except for the values we want to predict.
DT3 = CV.best_estimator_.fit(X_predict,y_predict)

Edit the cell below to change which compound is predicted between: Silicon, Silica, Salt, Diamond, and Tin

Change the Prediction_features object to one of the following:  
xpredict_Si  
xpredict_SiO2  
xpredict_NaCl  
xpredict_C  
xpredict_Sn  

In [ ]:
### MAKE EDITS BELOW HERE ###

Prediction_features = xpredict_Si

### MAKE EDITS ABOVE HERE ###

# make a prediction with the trained DT3 model
print("Predicted Band Gap: ",DT3.predict(Prediction_features))

Now for our final test on model performance. We are going to take the individual predictions on our Test data set and quantify how often the model succeeded or failed in making predictions for both the Solar application and Wide Band Gap application. Below we've rearranged the existing data from the parity plots in the previous section and printed it explicitly so we can look in more detail.

In doing this we are viewing the results of this regression model through the lens of classificaiton. Essentially the materials with known values in a certain range will be viewed as one class of materials, and everything else as another class. We'll then assess how well the model does at correctly identifying these classes of materials. If you want you read up on the background related to a few of these metrics you can look into the metrics precision and recall for binary classifiers. During the exercises below we'll walk through the process of calculating the recall for this pseudo-classification model.

In [ ]:
# combine previous data into one dataframe for visualization
predictions_combined = pd.DataFrame(list(zip(y_test,Test_predictions2)),columns=['test','predictions'])

In [ ]:
# sort on the Test values from low to high
predictions_combined.sort_values("test")

<div class = "exercise">

### Exercise 7.1<a name="e7.1"></a>
Questions:  

The first task was a use case where we want to predict single-junction solar materials and they need to have a band gap between 1.1 eV and 1.7 eV to have decent efficiency.  
1. In the Test dataset how many materials do we have with band gaps within the range of being a good solar material? Note in terms of classification we are identifying the number of positive cases in the dataset.  
2. Now we'll compare to the Predictions. Find the number of times the positive cases identified in question 1 are predicted to be single-junction solar materials (between 1.1 and 1.7 eV). This is referred to as the number of true positives, the number of times the positive cases were predicted correctly. Divide this number of true positives by the total number of positive cases (from question 1) to obtain the recall value. What is the recall of our pseudo-classifier to predict single-junction solar materials?

The second task was to identify high bandgap materials such as GaN, where the bandgap at at or above 3 eV.  
3. In the Test dataset how many materials do we have with band gaps at or above 3 eV?  
4. Perform the same process from question 2 (remember are classes are now defined differently for this new task) and calculate the recall for predicting high bandgap materials. What is the recall in this case?

Think about the performance of the model in correctly completing both of the prediction tasks on the test dataset.
5. Based on the evidence from questions 1-4 which tasks can the model succeed at?

Challenges (optional questions):
6. There are a number of other classification metrics we could have used to assess the performance of the model on the two potential tasks. Calculate the precision and false discovery rate for the two tasks. Do these metrics support the conclusion you made in question 5? We ignored this earlier in favor of simplicity, but potentially there is a more nuanced understanding of how the models might perform.

[Check or Submit Answers](#a7.1)

</div>

# The End

So, are we officially done? What's next in the machine learning workflow?

- hopefully you have your research problem down at this point. or else, figure out what you need to know, and whether ML can help with that
- go back and reiterate on hyperparameter tuning
- use a different model
    - get uncertainty estimate on your prediction by going from (decision) tree to (random) forest
- redo data cleaning/featurization
- get more data

what should you do next as a student?

if this is only interesting to you, and you don't plan to do ML yourself in the near future: solidify the big ideas and key takeaways.

if you want to get hands-on with ML:
- think about your data  
- go through the lab again and figure out each line of code  
- change parameters and do all the challenges  
- read the docs for software packages such as scikit-learn or mastml that help us perform these machine learning workflows.


 <div class = "answers">

### Answers<a name="answers"></a>

* [Back to TOC](#toc)


These are the answers for the code exercises.

</div>

#### [Back to Exercise 1.1](#e1.1) <a name="a1.1"></a>
1. There are duplicate entries for the same chemical formula  

#### [Back to Exercise 1.2](#e1.2) <a name="a1.2"></a>

1. 1447 data points  
2. 467 data points  

#### [Back to Exercise 1.3](#e1.3) <a name="a1.3"></a>
1. 13.591 eV  (13.096 eV if using the corrected averaging)  
2. 5 eV would be much too large to be a useful prediction, being more than twice the standard deviation of the data  
3. 0.5 eV would be a useful prediction error, being less than a quarter of the standard deviation in the dataset  

#### [Back to Exercise 1.4](#e1.4) <a name="a1.4"></a>
1. No, there is significantly more data between 0 and 1 eV  
2. No, making predictions above 5 eV might be unreasonable because we have so little data at the high end.  
3. We might consider throwing out some low bandgap data to make this more balanced.  

#### [Back to Exercise 1.5](#e1.5) <a name="a1.5"></a>
1. O, Se, S, Te, As  
2. Lu, Tm, Ir, Y, Ta  
3. Most confident in predictions of Oxides, then predictions containing Iridium, and least confident in predictions of an element that the model hasn't seen before 

#### [Back to Exercise 2.1](#e2.1) <a name="a2.1"></a>
1. 87 (scroll down to the bottom of the dataframe to see the nubmer of columns - each column represents a feature)

#### [Back to Exercise 2.2](#e2.2) <a name="a2.2"></a>
1. \begin{equation} \frac{3 * 3 + 51 * 1}{1 + 3} = 15 \end{equation}

#### [Back to Exercise 3.1](#e3.1) <a name="a3.1"></a>
1. 86  
2. No  

#### [Back to Exercise 3.2](#e3.2) <a name="a3.2"></a>
1. 71  
2. No (would be if very low like less than 10)

#### [Back to Exercise 4.1](#e4.1) <a name="a4.1"></a>
1. No the train and test splits are not identical, there may be some differences between train and test

#### [Back to Exercise 5.1](#e5.1) <a name="a5.1"></a>
1. 
    A. Yes, the predictions line up with the ideal line really well.  
    B. No.  
    C. No, there is no consistent deviation from the ideal line.  
2. Yes.  
3. Yes.

#### [Back to Exercise 5.2](#e5.2) <a name="a5.2"></a>
1.  
    A. Yes. The majority of predictions still lines up with the ideal line.  
    B. Yes, there are around 5-6 outliers spread out across the range of measured bandgaps.  
    C. No. Although there is more overall deviation from the ideal line, the predictions are spread out evenly on both sides of the ideal line across the entire range and show no consistent over/underpredicting.  
2. No. This use case requires that the prediction error doesn't exceed 0.3eV, but the test data RMSE is 1.2615 eV.  
3. Yes.

#### [Back to Exercise 5.3](#e5.3) <a name="a5.3"></a>
1. Better on training set, worse on test set.  
2. Overfit  
3. we need to use test data because the goal of ML, in our case, is to predict properties of materials that the model hasn't seen before, and test set performance is more representative of the prediction task.  

#### [Back to Exercise 5.4](#e5.4) <a name="a5.4"></a>
1. 10  
2. 1  
3. 1  
4. This supports the idea of the model being overfit as each training data point has it's own leaf node.

#### [Back to Exercise 6.1](#e6.1) <a name="a6.1"></a>

1. 50  
2. 50  
3. No, the model isn't changing as maximum leaf nodes increases beyond 20  
4. Yes, the model is increasing in complexity as we add more trees  
5. No, the model consistently improves in performance so it doesn't look like overfitting is increasing

#### [Back to Exercise 6.2](#e6.2) <a name="a6.2"></a>

1. Yes we see an improvement of 23.52%   
2. No.  
3. Yes.  

#### [Back to Exercise 6.3](#e6.3) <a name="a6.3"></a>

1. The optimized model is worse at predicting training data
2. No, test or validation predictions are necessary to estimate predictive performance

#### [Back to Exercise 6.4](#e6.4) <a name="a6.4"></a>

1. The optimized model does better at predicting test data.  
2. The difference between training and test performance decreased after hyperparameter optimization.  
3. Less overfit, between the training data is more similar to test data.  

#### [Back to Exercise 6.5](#e6.5) <a name="a6.5"></a>

1. In this lab, we use 3 main criteria for model selection:  
    1. Higher cross-validation score, which measures model prediction performance on unseen data  
    2. Better performance on test data, which also measures model prediction merformance on unseen data  
    3. Less overfit, which suggests the model is less likely to be adversely affected by spurious patterns in the training data  
    
   The optimized model has a higher CV score (see exercise 6.3), better performance on test data (exercise 6.4) and is less overfit (exercise 6.4) than the default model. We choose the optimized model because we expect it to have better final prediction performance based on the 3 criteria listed above.


#### [Back to Exercise 7.1](#e7.1) <a name="a7.1"></a>

1. 7 materials between 1.1 and 1.7 eV  
2. 1/7 of the materials is predicted in the range, ~14% of the time  
3. 9 materials at or above 3 eV  
4. 100% of materials are correctly predicted above 3 eV  
5. The second task of identifying wide bandgap materials the model succeeds at  
